# Diet Planning Optimization with Linear Programming

## Introduction

Designing an optimal diet plan is important for maintaining health and nutrition. This project aims to generate personalized diet plans using linear programming optimization. Generated plans are basically for **busy people** that need **minimum time to prepare and cook the foods**.

## What is a diet plan?

A diet plan specifies the types and amounts of food to eat over a set period of time. A good diet plan provides the right balance of calories, macronutrients (carbs, protein, and fat), vitamins and minerals needed for overall health and wellbeing.

### Why it's important?

* Maintain a healthy weight - A balanced diet can help achieve and maintain a healthy body weight.
* Meet nutrient needs - A well-planned diet ensures the intake of all essential nutrients required by the body.
* Improve health - Certain diets can help manage diseases like diabetes, high blood pressure, and high cholesterol.
* Increase energy levels - Nutritious foods fuel the body and improve physical and mental performance.

## Project Phases

This project aims to optimize diet plans with the following 5 phases:

- [ ] **[⛏ Data Gathering/Mining](#⛏Phase-1:-Data-Mining)** - Recipes and nutrition data will be scraped from the BBC Good Food website, focusing on quick and easy recipes that require minimal preparation time.
    - [x] Define what we are looking for
    - [x] Define data scrapping functions
    - [x] Store gathered data into dataset

- [x] **[🧹 Data Cleaning/Selecting](#🧹Phase-2:-Data-Cleaning)** - Missing and erroneous data will be identified and removed. Incomplete recipes will be discarded.
    - [x] Classify foods with category and sub-category
    - [x] Append recipes and their classification into dataset/data frame
    - [x] Remove missing values
    - [x] Convert mismatched data
    - [x] Select data by condition (Preparation time + Cocking time < 30 min)
    - [x] Save modified dataset into file (.csv)

- [x] **[⚖ Data Processing](#⚖Phase-3:-Data-Processing)** - An LP model will be formulated to generate optimized diet plans that maximize nutrition while minimizing preparation time.
    - [x] Make a structure for user prefrences input
    - [x] Find optimized mix of recipes according to `Cleaned Dataset`

- [x] **[✅ Validation](#✅Phase-4:-Validation)** - Results will be cross-checked against standards from the USDA food composition database to ensure nutritional adequacy.
    - [x] Check if found solution prefrences are according to user wants with respect to USDA Food Compositions
    - [x] Pass to Visualization or Recalculate solution
- [ ] **[📊 User Interface](#📊Phase-5:-Interface)** - A web app will be built to display personalized diet plans and the recipes included in each meal.

## ⛏Phase 1: Data Mining

Getting recipe data on [bbcgoodfood](https://www.bbcgoodfood.com/)

### Preparation of Packages

In the following code cell we are going to install and import the packages that we need to do datascraping.

In [15]:
!pip install beautifulsoup4 requests -q
import requests
from bs4 import BeautifulSoup as bs
import json

### Define Needed Data

Defining `base_url` and Getting the `data_urls` for all the recipes that we are intrested in.

In [16]:
base_url = "https://www.bbcgoodfood.com"

data_urls = {
    "recipes" : {
      'Lunch recipes': {
            'Quick lunch recipes':'https://www.bbcgoodfood.com/recipes/collection/quick-lunch-recipes',
            'Healthy lunch recipes': 'https://www.bbcgoodfood.com/recipes/collection/healthy-lunch-recipes',
            'Family lunch recipes': 'https://www.bbcgoodfood.com/recipes/collection/family-lunch-recipes',
            'Healthy packed lunches':'https://www.bbcgoodfood.com/recipes/collection/healthy-packed-lunch-recipes',
            'Dinner-to-lunch recipes': 'https://www.bbcgoodfood.com/recipes/collection/best-healthy-dinner-to-lunch-recipes'
      },
      'Dinner recipes':{
          'Chilli con carne recipes': 'https://www.bbcgoodfood.com/recipes/collection/chilli-con-carne-recipes',
          'Easy dinner recipes': 'https://www.bbcgoodfood.com/recipes/collection/quick-and-easy-recipes',
          'Pasta recipes': 'https://www.bbcgoodfood.com/recipes/collection/easy-pasta-recipes',
          'One pot recipes': 'https://www.bbcgoodfood.com/recipes/collection/storecupboard-one-pot-recipes',
          'Vegetarian dinners': 'https://www.bbcgoodfood.com/recipes/collection/healthy-vegetarian-dinner-recipes',
          'Vegan dinner recipes': 'https://www.bbcgoodfood.com/recipes/collection/healthy-vegan-recipes',
          'Low carb dinner recipes': 'https://www.bbcgoodfood.com/recipes/collection/low-carb-dinner-recipes',
      },
      'Breakfast recipes':{
          'Breakfast muffins': 'https://www.bbcgoodfood.com/recipes/collection/breakfast-muffin-recipes',
          'Breakfast pancakes': 'https://www.bbcgoodfood.com/recipes/collection/breakfast-pancake-recipes',
          'Overnight oat recipes': 'https://www.bbcgoodfood.com/recipes/collection/overnight-oats-recipes',
          'Breakfast smoothies': 'https://www.bbcgoodfood.com/recipes/collection/breakfast-smoothie-recipes',
          'Low calorie breakfasts': 'https://www.bbcgoodfood.com/recipes/collection/low-calorie-breakfast-recipes',
          'Mood boosting recipes': 'https://www.bbcgoodfood.com/recipes/collection/mood-boosting-breakfast-recipes',
          'Energy boosting recipes': 'https://www.bbcgoodfood.com/recipes/collection/energy-boosting-breakfast-recipes',
          'High fibre breakfasts': 'https://www.bbcgoodfood.com/recipes/collection/high-fibre-breakfast-recipes',
      },
      'Desserts':{
          "Easy dessert recipes": "https://www.bbcgoodfood.com/recipes/collection/quick-dessert-recipes",
          "Healthy dessert recipes": "https://www.bbcgoodfood.com/recipes/collection/healthy-desserts-recipes",
          "Apple dessert recipes": "https://www.bbcgoodfood.com/recipes/collection/apple-dessert-recipes",
          "Chocolate desserts": "https://www.bbcgoodfood.com/recipes/collection/chocolate-dessert-recipes",
          "Quick desserts": "https://www.bbcgoodfood.com/recipes/collection/quick-dessert-recipes",
          "Sweet tart recipes": "https://www.bbcgoodfood.com/recipes/collection/sweet-tart-recipes",
          "Easy pudding recipes": "https://www.bbcgoodfood.com/recipes/collection/chocolate-cake-recipes",
      },
      'Salads':{
          "Chicken salad recipes": "https://www.bbcgoodfood.com/recipes/collection/chicken-salad-recipes",
          "Green salad recipes": "https://www.bbcgoodfood.com/recipes/collection/green-salad-recipes",
          "Lentil salad recipes": "https://www.bbcgoodfood.com/recipes/collection/lentil-salad-recipes",
          "Warm salad dressings": "https://www.bbcgoodfood.com/recipes/collection/warm-salad-recipes",
        }
    },
    "inspiration": {
        'Coffees':{
            "Irish coffee recipes": "https://www.bbcgoodfood.com/recipes/irish-coffee",
            "Iced coffee recipe": "https://www.bbcgoodfood.com/recipes/iced-coffee",
            "Cold coffee recipe": "https://www.bbcgoodfood.com/recipes/cold-brew-coffee",
            "Dalgona coffee recipe": "https://www.bbcgoodfood.com/recipes/dalgona-coffee",
        },
        'Teas':{
            "Bubble tea recipe": "https://www.bbcgoodfood.com/recipes/bubble-tea",
            "Iced tea recipe": "https://www.bbcgoodfood.com/recipes/iced-tea",
            "Lemon tea recipe": "https://www.bbcgoodfood.com/recipes/lemon-ginger-tea",
            "Mint tea recipe": "https://www.bbcgoodfood.com/recipes/fresh-mint-tea",
        }
    }
}

### Define Functions

Helper function that get urls from each category called `get_urls` and Other functions that allow us to scrap data on the pages.

#### get_urls()

In [17]:
def get_urls(url:str) -> list[str]:
  html = requests.get(url)
  soup = bs(html.content, 'html.parser')
  anchors = soup.find('ul', {'class': "dynamic-list__list list"}).find_all('a', {'class': 'link d-block'})
  return [base_url + a['href'] for a in anchors]

#### getNutrients()

In [18]:
def getNutrients(tables):
  nutri = list()
  for table in tables:
    trs = table.find_all('tr')
    for tr in trs:
      tds = [t.text for t in  tr.find_all('td')[1:]]
      if len(tds) == 2:
        nutri.append(tds)
  return dict(nutri)

#### getTimes()

In [19]:
def getTimes(times):
  data = list()
  for i, time in enumerate(times):
    span = [s for s in time.find_all('span')]
    if i == 0:
      data.append(['Preparation', span[1].text if len(span) ==2 else "No Time"])
    elif i == 1:
      data.append(['Cooking', span[1].text if len(span) ==2 else "No Time"])
  return dict(data)

#### scrapeData()

And finally, the `scrapeData` function that scrap data on the page given a url using [get_urls()](#get_urls()), [getNutrients()](#getNutrients()) and [getTimes()](#getTimes())

In [20]:
def scrapeData(url:str):
  try:
    html = requests.get(url)
    soup = bs(html.content, 'html.parser')
    # getting the image
    img = soup.find('div', {'class': 'image__container'}).find('img', {'class': 'image__img'})['src'].split('?')[0]
    # getting the name
    name = soup.find('h1', {'class': 'heading-1'}).text
    # getting description
    description = soup.find('div', {'class': 'editor-content mt-sm pr-xxs hidden-print'}).find('p').text
    ingredients = [i.text for i in soup.find('section', {'class': 'recipe__ingredients col-12 mt-md col-lg-6'}).find_all('li')]
    steps = [i.text for i in soup.find('div', {'class': 'row recipe__instructions'}).find_all('div', {'class': 'editor-content'})]
    nutri_tables = [table for table in soup.find_all('tbody', {'class': 'key-value-blocks__batch body-copy-extra-small'})]
    nutrients = getNutrients(nutri_tables)
    _times = soup.find('div', {'class': 'icon-with-text time-range-list cook-and-prep-time post-header__cook-and-prep-time'}).find_all('li')
    times = getTimes(_times)
    return {
        'url': url,
        'image': img,
        'name': name,
        'description': description,
        'ingredients': ingredients,
        'steps': steps,
        'nutrients': nutrients,
        'times': times
    }
  except:
    return None

#Test Functions
scrapeData('https://www.bbcgoodfood.com/recipes/thai-chicken-salad')

{'url': 'https://www.bbcgoodfood.com/recipes/thai-chicken-salad',
 'image': 'https://images.immediate.co.uk/production/volatile/sites/30/2020/08/thai-chicken-fa2a73b.jpg',
 'name': 'Thai chicken salad',
 'description': "This salad couldn't be easier to make - simply shred leftover cooked chicken and mix with Asian flavours, tropical fruit, cashew nuts and herbs",
 'ingredients': ['1 head Chinese leaf, shredded',
  '2 cooked chicken breasts or 200g leftover cooked chicken, shredded',
  '1 mango, peeled, stoned and thinly sliced',
  'bunch mint, leaves picked',
  '6 spring onions, sliced diagonally',
  '3 tbsp salted peanut or cashew nuts, roughly chopped',
  'juice 4 limes',
  '4 tbsp sesame oil',
  'pinch of sugar',
  'splash of  fish sauce',
  '2 large red chillies, deseeded and finely chopped'],
 'steps': ['To make the dressing, mix together all the ingredients and stir to dissolve the sugar.',
  'In a large bowl, mix all the salad ingredients except the nuts. Toss with the dressing 

#### getExampleCategory, getSubCategoryData & getMainCategoryData

Helper functions that allows us to collect data from each url.

In [21]:
def getExampleCategory(examples):
  data = list()
  for key, url in examples.items():
    urls = get_urls(url) if url.find('collection') != -1 else [url]
    values = list()
    for u in urls:
      v = scrapeData(u)
      if v is not None:
        values.append(v)
    data.append([key, values ])
  return dict(data)

def getSubCategoryData(categories, main_category):
  data = list()
  for sub_category in categories.keys():
    examples = data_urls[main_category][sub_category]
    data.append([sub_category, getExampleCategory(examples)])
  return dict(data)

def getMainCategoryData(urls):
  data = list()
  for main_category in urls.keys():
    cates = data_urls[main_category]
    data.append([main_category, getSubCategoryData(cates, main_category)])
  return dict(data)

### Gathering the data

Saving the data as `json` file in a file named `recipes.json`

**I had to use `proxy` due to my country's access restriction on [bbcgoodfood](https://www.bbcgoodfood.com/)**

In [22]:
json_data = getMainCategoryData(data_urls)
with open('recipes.json', 'w') as writter:
  writter.write(json.dumps(json_data, indent=2))

## 🧹Phase 2: Data Cleaning

The most important part of this project from my aspect of view is this part. a clean and selected dataset is basically the heart of any analyzation algorithm; Without clean data, even the best algorithms fails.

### Data Preprocessing
First of all, we need to load our data into data frame. as columns name are mix of category and sub-category, we must separate them and use them as an classifier for each food instead of columns name.

In [29]:
import pandas as pd

data = json.load(open("recipes.json"))
df = pd.json_normalize(data)
temp=[]
sum=0
for col in df.columns:
    c= col.split(".")[1:]
    for i in range(len(df[col][0])):
        temp+=[[c,df[col][0][i]]]
data=[]
for sublist in temp:
    c1,c2=sublist[0]
    dictionary=sublist[1]
    new_dict={'Main Category':c1,'Sub-Category':c2, **dictionary}
    data.append(new_dict)

df=pd.DataFrame(data)
df

,Main Category,Sub-Category,url,image,name,description,ingredients,steps,nutrients,times
0,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/smoked-sal...,https://images.immediate.co.uk/production/vola...,"Smoked salmon, quinoa & dill lunch pot",This easy packed lunch is as delicious as it i...,"[2 tbsp half-fat soured cream, 2 tbsp lemon ju...","[First, make the dressing. Mix the soured crea...","{'kcal': '254', 'fat': '7g', 'saturates': '2g'...","{'Preparation': '15 mins', 'Cooking': 'No Time'}"
1,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/air-fryer-...,https://images.immediate.co.uk/production/vola...,Air fryer chicken breasts,Use an air fryer to create this tempting dish ...,"[4 chicken breasts, ½ tbsp rapeseed oil, 1 ts...",[Coat the chicken in the oil and set aside. In...,{},"{'Preparation': '5 mins', 'Cooking': '18 mins ..."
2,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/panuozzo-s...,https://images.immediate.co.uk/production/vola...,Panuozzo sandwich,Make your own baguettes and pesto to make thes...,"[300g strong white bread flour, plus extra for...","[Put the flour in a large bowl, then stir in t...",{},"{'Preparation': '20 mins', 'Cooking': '12 mins'}"
3,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/bulgur-qui...,https://images.immediate.co.uk/production/vola...,Bulgur & quinoa lunch bowls,These meal prep grain bowls use one base and t...,"[1 large onion, very finely chopped, 150g bulg...","[Tip the onion and bulgur mix into a pan, pour...","{'kcal': '369', 'fat': '20g', 'saturates': '4g...","{'Preparation': '5 mins', 'Cooking': '15 mins'}"
4,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/falafel-bu...,https://images.immediate.co.uk/production/vola...,Falafel burgers,A healthy burger that's filling too. These are...,"[400g can chickpeas, rinsed and drained, 1 sm...",[Drain the chickpeas and pat dry with kitchen ...,"{'kcal': '161', 'fat': '8g', 'saturates': '1g'...","{'Preparation': '10 mins', 'Cooking': '6 mins'}"
...,...,...,...,...,...,...,...,...,...,...
728,Coffees,Dalgona coffee recipe,https://www.bbcgoodfood.com/recipes/dalgona-co...,https://images.immediate.co.uk/production/vola...,Dalgona coffee,Our easy whipped coffee recipe is simple enoug...,"[3 tbsp instant coffee, 2 tbsp sugar, 400-500m...","[Whisk the coffee, sugar and 3 tbsp boiling wa...",{},{'Preparation': '8 mins'}
729,Teas,Bubble tea recipe,https://www.bbcgoodfood.com/recipes/bubble-tea,https://images.immediate.co.uk/production/vola...,How to make bubble tea,Try a Taiwanese favourite – homemade brown sug...,"[2 teabags or 2g black tea, 50g caster sugar, ...",[Put the teabags or tea leaves in 100ml boilin...,{},"{'Preparation': '35 mins', 'Cooking': 'No Time'}"
730,Teas,Iced tea recipe,https://www.bbcgoodfood.com/recipes/iced-tea,https://images.immediate.co.uk/production/vola...,Easy iced tea,Make up a jug of refreshing iced tea to serve ...,"[6 tea bags, 2 tbsp golden caster sugar, 1 tbs...","[Put the tea bags, sugar, honey and 1.5 litres...",{},"{'Preparation': '5 mins', 'Cooking': 'No Time'}"
731,Teas,Lemon tea recipe,https://www.bbcgoodfood.com/recipes/lemon-ging...,https://images.immediate.co.uk/production/vola...,Lemon & ginger tea,Combine lemon with root ginger to make this re...,"[1 lemon, 2cm piece root ginger, finely sliced...",[Cut the lemon in half. Squeeze the juice from...,{},{'Preparation': '5 mins'}


### Handeling Missing/Mismatched Value

As data in this dataset gathered analytically with experiments and due to data sensitivity, filling missing value with statistical parameter (like `mean`) cause significant error. so, I decide to remove missing values instead. Also as data specially in `nutrients` and `times` column is string, we must change them into float and int so we could operate calculation on them.(times value used maximum amount of time not a range of time)

In [32]:
import re

def convert_nutrients_values(d):
    # Remove any rows with empty dictionary
    if not d:
        return None
    # Convert string values to float values
    for k, v in d.items():
        if not v:
            d[k] = None
        else:
            try:
                if isinstance(v, str) and v[-1] == 'g':
                    d[k] = float(v[:-1])
                else:
                    d[k] = float(v)
            except ValueError:
                d[k] = None
    check=False
    for k, _ in d.items():
      if d[k]!=None and d[k]!=0.0:
        check=True
        return d
    return None

def convert_times_values(d):
    # Convert time strings to minutes
    preparation_time = d.get('Preparation', '0 mins')
    cooking_time = d.get('Cooking', '0 mins')
    preparation_minutes = int(re.findall('\d+', preparation_time)[0]) if re.findall('\d+', preparation_time) else 0
    cooking_minutes = int(re.findall('\d+', cooking_time)[0]) if re.findall('\d+', cooking_time) else 0
    if '-' in cooking_time:
        cooking_range = re.findall('\d+', cooking_time)
        cooking_minutes = int(max(cooking_range))
    if preparation_minutes + cooking_minutes > 0:
      return preparation_minutes + cooking_minutes
    return None

df['nutrients'] = df['nutrients'].apply(convert_nutrients_values)
df['times'] = df['times'].apply(convert_times_values)

In [33]:
df = df.dropna().reset_index(drop=True)
df

,Main Category,Sub-Category,url,image,name,description,ingredients,steps,nutrients,times
0,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/smoked-sal...,https://images.immediate.co.uk/production/vola...,"Smoked salmon, quinoa & dill lunch pot",This easy packed lunch is as delicious as it i...,"[2 tbsp half-fat soured cream, 2 tbsp lemon ju...","[First, make the dressing. Mix the soured crea...","{'kcal': 254.0, 'fat': 7.0, 'saturates': 2.0, ...",15
1,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/bulgur-qui...,https://images.immediate.co.uk/production/vola...,Bulgur & quinoa lunch bowls,These meal prep grain bowls use one base and t...,"[1 large onion, very finely chopped, 150g bulg...","[Tip the onion and bulgur mix into a pan, pour...","{'kcal': 369.0, 'fat': 20.0, 'saturates': 4.0,...",20
2,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/falafel-bu...,https://images.immediate.co.uk/production/vola...,Falafel burgers,A healthy burger that's filling too. These are...,"[400g can chickpeas, rinsed and drained, 1 sm...",[Drain the chickpeas and pat dry with kitchen ...,"{'kcal': 161.0, 'fat': 8.0, 'saturates': 1.0, ...",16
3,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/spicy-chic...,https://images.immediate.co.uk/production/vola...,Spicy chicken & avocado wraps,"Pan-fry lean chicken breast with lime, chilli ...","[1 chicken breast (approx 180g), thinly sliced...","[Mix the chicken with the lime juice, chilli p...","{'kcal': 403.0, 'fat': 16.0, 'saturates': 4.0,...",13
4,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/prawn-mang...,https://images.immediate.co.uk/production/vola...,Prawn & mango salad,Kids will love this colourful salad with avoca...,"[½ avocado, peeled and cut into cubes, see tip...","[Mix the avocado with the lemon juice, then to...","{'kcal': 168.0, 'fat': 9.0, 'saturates': 2.0, ...",10
...,...,...,...,...,...,...,...,...,...,...
397,Salads,Warm salad dressings,https://www.bbcgoodfood.com/recipes/warm-thai-...,https://images.immediate.co.uk/production/vola...,Warm Thai chicken & noodle salad,"A low fat, oriental-inspired salad - ideal for...","[2 large skinless chicken breasts, 175g dried...",[Preheat the grill to high. Put the chicken on...,"{'kcal': 336.0, 'fat': 10.0, 'saturates': 1.0,...",45
398,Salads,Warm salad dressings,https://www.bbcgoodfood.com/recipes/warm-sausa...,https://images.immediate.co.uk/production/vola...,Warm sausage & broccoli pasta salad,A simple midweek supper of short pasta flavour...,"[200g Cumberland pork chipolata, 1 tbsp olive ...","[Heat the grill, and cook the sausages for 10-...","{'kcal': 383.0, 'fat': 17.0, 'saturates': 5.0,...",35
399,Coffees,Iced coffee recipe,https://www.bbcgoodfood.com/recipes/iced-coffee,https://images.immediate.co.uk/production/vola...,Iced coffee,Discover how to make iced coffee at home. With...,"[200ml strong black coffee, 50ml milk, ice, ma...",[Make a 200ml cup of black coffee following pa...,"{'kcal': 27.0, 'fat': 1.0, 'saturates': 0.5, '...",5
400,Coffees,Cold coffee recipe,https://www.bbcgoodfood.com/recipes/cold-brew-...,https://images.immediate.co.uk/production/vola...,How to make cold brew coffee,"High in caffeine, this iced cold brew coffee w...",[50g ground coffee (use coarsely ground for ca...,[Put 400ml cold water into a large jug or jar ...,"{'kcal': 0.0, 'fat': 0.0, 'saturates': 0.0, 'c...",5


### Data Selection/ Storing

As diet plan is for **busy people**, I decided to choose `times < 30`

Save the final Dataset into `processed_data.csv`

In [34]:
df=df[df['times'] <= 30]
df.to_csv('processed_data.csv', index=False)
df

,Main Category,Sub-Category,url,image,name,description,ingredients,steps,nutrients,times
0,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/smoked-sal...,https://images.immediate.co.uk/production/vola...,"Smoked salmon, quinoa & dill lunch pot",This easy packed lunch is as delicious as it i...,"[2 tbsp half-fat soured cream, 2 tbsp lemon ju...","[First, make the dressing. Mix the soured crea...","{'kcal': 254.0, 'fat': 7.0, 'saturates': 2.0, ...",15
1,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/bulgur-qui...,https://images.immediate.co.uk/production/vola...,Bulgur & quinoa lunch bowls,These meal prep grain bowls use one base and t...,"[1 large onion, very finely chopped, 150g bulg...","[Tip the onion and bulgur mix into a pan, pour...","{'kcal': 369.0, 'fat': 20.0, 'saturates': 4.0,...",20
2,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/falafel-bu...,https://images.immediate.co.uk/production/vola...,Falafel burgers,A healthy burger that's filling too. These are...,"[400g can chickpeas, rinsed and drained, 1 sm...",[Drain the chickpeas and pat dry with kitchen ...,"{'kcal': 161.0, 'fat': 8.0, 'saturates': 1.0, ...",16
3,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/spicy-chic...,https://images.immediate.co.uk/production/vola...,Spicy chicken & avocado wraps,"Pan-fry lean chicken breast with lime, chilli ...","[1 chicken breast (approx 180g), thinly sliced...","[Mix the chicken with the lime juice, chilli p...","{'kcal': 403.0, 'fat': 16.0, 'saturates': 4.0,...",13
4,Lunch recipes,Quick lunch recipes,https://www.bbcgoodfood.com/recipes/prawn-mang...,https://images.immediate.co.uk/production/vola...,Prawn & mango salad,Kids will love this colourful salad with avoca...,"[½ avocado, peeled and cut into cubes, see tip...","[Mix the avocado with the lemon juice, then to...","{'kcal': 168.0, 'fat': 9.0, 'saturates': 2.0, ...",10
...,...,...,...,...,...,...,...,...,...,...
393,Salads,Warm salad dressings,https://www.bbcgoodfood.com/recipes/steak-beet...,https://images.immediate.co.uk/production/vola...,"Steak, beetroot, horseradish & warm lentil salad",Get four of your 5-a-day in one meal from this...,"[1 tbsp hot horseradish sauce, 2 tbsp Greek yo...","[Whisk together the horseradish, yogurt and ho...","{'kcal': 496.0, 'fat': 13.0, 'saturates': 4.0,...",20
395,Salads,Warm salad dressings,https://www.bbcgoodfood.com/recipes/warm-chick...,https://images.immediate.co.uk/production/vola...,Warm chicken salad with garlic mushrooms,"A mouthwatering, warm winter salad made from j...","[2 large chicken legs or 4 large thighs, 2 tbs...",[Preheat the oven to 220C/Gas 7/fan oven 200C....,"{'kcal': 385.0, 'fat': 30.0, 'saturates': 7.0,...",11
399,Coffees,Iced coffee recipe,https://www.bbcgoodfood.com/recipes/iced-coffee,https://images.immediate.co.uk/production/vola...,Iced coffee,Discover how to make iced coffee at home. With...,"[200ml strong black coffee, 50ml milk, ice, ma...",[Make a 200ml cup of black coffee following pa...,"{'kcal': 27.0, 'fat': 1.0, 'saturates': 0.5, '...",5
400,Coffees,Cold coffee recipe,https://www.bbcgoodfood.com/recipes/cold-brew-...,https://images.immediate.co.uk/production/vola...,How to make cold brew coffee,"High in caffeine, this iced cold brew coffee w...",[50g ground coffee (use coarsely ground for ca...,[Put 400ml cold water into a large jug or jar ...,"{'kcal': 0.0, 'fat': 0.0, 'saturates': 0.0, 'c...",5


## ⚖Phase 3: Data Processing

And finally the Optimization part! now as we have all data we need, it's time to start generating optimal diet plan using linear programming library: `PuLP`

### Prepare requirements

In [61]:
!pip install pulp

from pulp import *
import pandas as pd

Max_kCal = 400
Min_Fib = 10
Min_Pro = 10
Max_time = 120

# Load the dataset
df = pd.read_csv('processed_data.csv')

### LP Model

Documentation: [PuLP 2.7]("https://coin-or.github.io/pulp/index.html")

Example: [Hands-On Linear Programming]("https://realpython.com/linear-programming-python/")

In [63]:
# Define the decision variables
foods = df['name'].tolist()
Plan = dict.fromkeys(df['Main Category'].tolist())
x = LpVariable.dicts('x', range(len(foods)), cat='Binary')
# Define the problem
prob = LpProblem('Diet Optimization', LpMaximize)
# # Define the objective function
prob += lpSum([x[i] for i in range(len(foods))])

# Define the constraints
prob += lpSum([x[i] * eval(df['nutrients'][i])['kcal'] for i in range(len(foods))]) <= Max_kCal
prob += lpSum([x[i] * eval(df['nutrients'][i])['fibre'] for i in range(len(foods))]) >= Min_Fib
prob += lpSum([x[i] * eval(df['nutrients'][i])['protein'] for i in range(len(foods))]) >= Min_Pro
# prob += lpSum([x[i] * eval(df['times'][i]) for i in range(len(foods))]) <= Max_time

# Add the category constraints: Make sure that breakfast,lunch and dinner is not empty
categories = ['Breakfast recipes', 'Lunch recipes', 'Dinner recipes', 'Coffees', 'Teas']
for c in categories:
    prob += lpSum([x[i] for i in range(len(foods)) if c in df.loc[df['name'] == foods[i], 'Main Category'].values]) >= 1
    # prob += lpSum([x[i] for i in range(len(foods)) if c in df.loc[df['name'] == foods[i], 'Main Category'].values]) <= 3

# for c in Plan.keys():
#     prob += lpSum([x[i] for i in range(len(foods)) if c in df.loc[df['name'] == foods[i], 'Main Category'].values]) <= 5
# Solve the problem
prob.solve()

# Initiate the Plan print
for cat in Plan.keys():
  Plan[cat]=[]
print('Status:', LpStatus[prob.status])
print('# Food :', value(prob.objective))
for i in range(len(foods)):
    if x[i].value() == 1:
      for Cat in Plan.keys():
        if df['Main Category'][i] == Cat:
          Plan[Cat].append(df['name'][i])

# make array size of dict's value as the same size as each other (Fill null cell)
def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

#Make final result
Plan = pad_dict_list(Plan, '---')
Plan=pd.DataFrame(Plan)

#Show Result
Plan[[filled for filled in Plan.columns if Plan[filled][0] != '---']]

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
# Food : 9.0


,Lunch recipes,Dinner recipes,Breakfast recipes,Salads,Coffees,Teas
0,Healthy chicken salad,Chickpea salad,Easy pancakes,Green salad with buttermilk dressing,Iced coffee,Fresh mint tea
1,---,---,---,Green goddess salad,How to make cold brew coffee,---
2,---,---,---,Shredded greens salad,---,---


## ✅Phase 4: Validation

## 📊Phase 5: Interface